In [1]:
# --- 1. Импорты и общие настройки ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet  # pip install prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")
# plt.style.use("seaborn-whitegrid")



In [2]:
# --- 2. Загрузка данных ---
# df должен содержать столбцы: "Регион", "Период" (YYYY-MM), и целевой столбец с показателем.
df = pd.read_excel("Датасет по яйцам.xlsx")
df["Период"] = pd.to_datetime(df["Период"], format="%Y-%m")
df.sample(10)



,Регион,Период,Яйца
1431,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2017-09-01,849.70
1239,КАРАГАНДИНСКАЯ ОБЛАСТЬ,2021-09-01,56291.70
99,АКМОЛИНСКАЯ ОБЛАСТЬ,2023-04-01,56116.40
1759,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2017-04-01,13336.20
1079,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2018-05-01,15614.30
1278,КАРАГАНДИНСКАЯ ОБЛАСТЬ,2024-12-01,51088.60
1767,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2017-12-01,15295.50
1883,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2017-08-01,73107.00
1019,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2023-05-01,9513.80
216,АКТЮБИНСКАЯ ОБЛАСТЬ,2023-01-01,19657.80


In [3]:
regions = df['Регион'].unique()
target   = "Яйца"
horizon  = 3
epsilon = 1e-6

In [4]:
first_test = pd.to_datetime("2024-01-01")
last_possible = df["Период"].max() - pd.DateOffset(months=horizon-1)
test_starts = pd.date_range(first_test, last_possible, freq="MS")

## Holw-Winter's (log)

In [5]:
results_hw = []
for region in regions:
    ts = (df[df["Регион"] == region]
          .set_index("Период")[target]
          .dropna()
          .sort_index())
    if len(ts) < 24:
        print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        # формируем train / test
        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

        # пропускаем, если недостаточно данных или неполный test
        if len(train) < 24 or len(test) < horizon:
            continue

        # обучаем модель
        train_log = np.log1p(train)

        hw_log = ExponentialSmoothing(
            train_log,
            seasonal="add",
            seasonal_periods=12
        ).fit(optimized=True)

        # прогноз и метрики
        fc_log = hw_log.forecast(horizon)
        fc = np.expm1(fc_log) 
        # fc   = model.forecast(horizon)
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test - fc) / test).mean()) * 100

        results_hw.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "Forecast":    [x.round(2) for x in list(fc.values)],
            "Actual":      [y.round(2) for y in list(test.values)],
            "RMSE":        rmse,
            "MAE":         mae,
            "MAPE_%":      mape
        })

# 4) Усреднение по всем скользящим окнам для каждого региона
res_hw = pd.DataFrame(results_hw)
print("Результаты прогнозов HW на 3 месяца:")

display(res_hw)

final_hw = (
    res_hw
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Holt–Winter's по регионам (rolling-3):")
display(final_hw)

Результаты прогнозов HW на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[50685.04, 48891.84, 53982.82]","[46739.2, 48287.8, 64167.2]",6315.49,4911.42,8.52
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[46756.38, 51642.25, 51453.59]","[48287.8, 64167.2, 59663.6]",8691.45,7422.13,12.15
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[52569.79, 52369.71, 58457.25]","[64167.2, 59663.6, 60711.6]",8016.29,7048.55,11.34
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[58329.06, 65051.12, 65612.23]","[59663.6, 60711.6, 60767.6]",3833.31,3506.23,5.79
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[65898.11, 66458.32, 65057.1]","[60711.6, 60767.6, 64270.3]",4468.53,3888.01,6.38
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[22454.31, 20977.66, 21074.61]","[22349.7, 22437.4, 23808.0]",1790.08,1432.58,6.15
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[20880.32, 20977.83, 19135.77]","[22437.4, 23808.0, 22847.1]",2840.68,2699.53,11.69
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[22522.89, 20527.18, 19713.57]","[23808.0, 22847.1, 21639.9]",1892.47,1843.79,8.15
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[21683.66, 20809.39, 18639.91]","[22847.1, 21639.9, 19659.4]",1013.69,1004.48,4.71


Средние метрики Holt–Winter's по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,4310.41,3781.74,6.27
1,АКТЮБИНСКАЯ ОБЛАСТЬ,1190.36,1068.68,5.93
2,АЛМАТИНСКАЯ ОБЛАСТЬ,3728.68,3296.67,7.25
3,АТЫРАУСКАЯ ОБЛАСТЬ,569.80,519.30,18.92
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,271.45,244.39,4.91
5,ГАЛМАТЫ,4.51,3.06,19.22
6,ГАСТАНА,0.27,0.24,144.40
7,ГШЫМКЕНТ,1063.23,896.09,5.94
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2032.78,1861.03,17.52
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,936.54,856.49,6.87


In [6]:
res_hw[res_hw['Регион'] == 'ГАСТАНА']

,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
60,ГАСТАНА,2024-01,2024-03,"[0.0, 0.0, 0.13]","[0.2, 0.2, 0.2]",0.17,0.16,78.07
61,ГАСТАНА,2024-02,2024-04,"[0.18, 0.32, 1.0]","[0.2, 0.2, 0.2]",0.47,0.31,155.97
62,ГАСТАНА,2024-03,2024-05,"[0.34, 1.02, 0.88]","[0.2, 0.2, 0.2]",0.62,0.55,273.14
63,ГАСТАНА,2024-04,2024-06,"[0.83, 0.7, 0.69]","[0.2, 0.2, 0.2]",0.54,0.54,270.52
64,ГАСТАНА,2024-05,2024-07,"[0.12, 0.12, -0.07]","[0.2, 0.2, 0.2]",0.17,0.14,71.83
65,ГАСТАНА,2024-06,2024-08,"[0.19, -0.0, 0.05]","[0.2, 0.2, 0.1]",0.12,0.09,50.64
66,ГАСТАНА,2024-07,2024-09,"[-0.0, 0.06, 0.03]","[0.2, 0.1, 0.1]",0.13,0.11,72.22
67,ГАСТАНА,2024-08,2024-10,"[0.25, 0.21, 0.03]","[0.1, 0.1, 0.1]",0.11,0.11,110.37
68,ГАСТАНА,2024-09,2024-11,"[0.08, -0.08, -0.17]","[0.1, 0.1, 0.1]",0.19,0.16,157.95
69,ГАСТАНА,2024-10,2024-12,"[-0.07, -0.15, -0.09]","[0.1, 0.1, 0.1]",0.21,0.20,203.34


## SARIMA

In [7]:
results_sarima = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12

        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            D=1 if use_seasonal else 0,      # фиксируем порядок сезонной разности
            seasonal_test=None,               # пропустить nsdiffs
            boxcox=True,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
      
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         [round(y,2) for y in actual]
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

Результаты прогнозов SARIMA на 3 месяца:


,Регион,Test start,Test end,order,seasonal_order,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"(1, 0, 1)","(2, 1, 1, 12)",5051.70,4697.70,8.61,"[51074.36, 50913.26, 57034.71]","[46739.2, 48287.8, 64167.2]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"(1, 0, 1)","(0, 1, 2, 12)",7628.01,6410.64,10.45,"[47185.39, 52980.57, 52720.73]","[48287.8, 64167.2, 59663.6]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"(1, 0, 5)","(2, 1, 1, 12)",7107.01,5837.03,9.35,"[53302.02, 53953.95, 59775.33]","[64167.2, 59663.6, 60711.6]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"(1, 0, 3)","(0, 1, 1, 12)",5608.71,4964.34,8.19,"[57769.02, 65435.48, 69042.15]","[59663.6, 60711.6, 60767.6]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"(1, 0, 3)","(0, 1, 1, 12)",6951.16,6687.66,10.86,"[66599.09, 70071.33, 69142.06]","[60711.6, 60767.6, 64270.3]"
...,...,...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"(0, 0, 1)","(0, 1, 1, 12)",1026.06,833.41,3.64,"[23849.25, 22485.5, 24760.58]","[22349.7, 22437.4, 23808.0]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"(1, 0, 1)","(0, 1, 0, 12)",2070.28,1882.53,8.18,"[21410.26, 25366.95, 25908.61]","[22437.4, 23808.0, 22847.1]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"(1, 0, 1)","(0, 1, 0, 12)",2489.35,2124.99,9.17,"[26434.14, 26249.11, 21986.71]","[23808.0, 22847.1, 21639.9]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"(1, 0, 1)","(0, 1, 1, 12)",663.37,559.32,2.73,"[22614.81, 21249.63, 18603.99]","[22847.1, 21639.9, 19659.4]"


Средние метрики SARIMA по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,4533.82,4015.63,6.72
1,АКТЮБИНСКАЯ ОБЛАСТЬ,1278.08,1138.98,6.20
2,АЛМАТИНСКАЯ ОБЛАСТЬ,5477.22,5060.42,11.34
3,АТЫРАУСКАЯ ОБЛАСТЬ,1424.04,1302.33,60.52
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,388.40,348.57,6.32
5,ГАЛМАТЫ,3.69,2.59,16.11
6,ГАСТАНА,0.14,0.13,89.75
7,ГШЫМКЕНТ,1539.50,1373.40,8.99
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2114.02,1941.30,18.91
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,965.09,859.96,6.90


## Facebook Prophet

In [8]:
results_prophet = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. данных, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # Подготовка данных для Prophet
#         df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
# # подготовка для одного региона
        df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
        df_prophet["y"] = np.log(df_prophet["y"] + epsilon)

        m = Prophet()
        m.fit(df_prophet)

        future = m.make_future_dataframe(periods=horizon, freq="MS")
        forecast = m.predict(future)

        # берем только прогнозные точки
        yhat_log = forecast["yhat"].values[-horizon:]
        fc = np.exp(yhat_log) - epsilon

        # m = Prophet()
        # m.fit(df_prophet)

        # # Создаем DataFrame будущих дат и делаем прогноз
        # # future = m.make_future_dataframe(periods=horizon, freq="MS")
        # # forecast = m.predict(future)

        # # Отбираем только наши горизонты
        # fc = forecast.set_index("ds")["yhat"].loc[test.index].values
        actual = test.values

        # Расчет метрик
        rmse  = np.sqrt(mean_squared_error(actual, fc))
        mae   = mean_absolute_error(actual, fc)
        mape  = (np.abs((actual - fc) / actual).mean()) * 100

        results_prophet.append({
            "Регион":     region,
            "Test start": test_start.strftime("%Y-%m"),
            "Test end":   test_end.strftime("%Y-%m"),
            "RMSE":       round(rmse, 2),
            "MAE":        round(mae, 2),
            "MAPE_%":     round(mape, 2),
            "Forecast":   [round(x, 2) for x in fc],
            "Actual":     [round(x, 2) for x in actual]
        })

# Собираем результаты в DataFrame
res_prophet = pd.DataFrame(results_prophet)
print("Результаты прогнозов Prophet на 3 месяца:")
display(res_prophet)

final_prophet = (
    res_prophet
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Prophet по регионам (rolling-3):")
display(final_prophet)


18:09:30 - cmdstanpy - INFO - Chain [1] start processing
18:09:31 - cmdstanpy - INFO - Chain [1] done processing
18:09:31 - cmdstanpy - INFO - Chain [1] start processing
18:09:31 - cmdstanpy - INFO - Chain [1] done processing
18:09:31 - cmdstanpy - INFO - Chain [1] start processing
18:09:31 - cmdstanpy - INFO - Chain [1] done processing
18:09:32 - cmdstanpy - INFO - Chain [1] start processing
18:09:32 - cmdstanpy - INFO - Chain [1] done processing
18:09:32 - cmdstanpy - INFO - Chain [1] start processing
18:09:32 - cmdstanpy - INFO - Chain [1] done processing
18:09:32 - cmdstanpy - INFO - Chain [1] start processing
18:09:32 - cmdstanpy - INFO - Chain [1] done processing
18:09:33 - cmdstanpy - INFO - Chain [1] start processing
18:09:33 - cmdstanpy - INFO - Chain [1] done processing
18:09:33 - cmdstanpy - INFO - Chain [1] start processing
18:09:33 - cmdstanpy - INFO - Chain [1] done processing
18:09:33 - cmdstanpy - INFO - Chain [1] start processing
18:09:33 - cmdstanpy - INFO - Chain [1]

Результаты прогнозов Prophet на 3 месяца:


,Регион,Test start,Test end,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,5469.43,5068.56,9.23,"[50733.78, 51554.24, 56222.54]","[46739.2, 48287.8, 64167.2]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,6215.98,5769.11,9.72,"[51229.08, 55557.27, 53907.47]","[48287.8, 64167.2, 59663.6]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,6612.81,6091.37,9.82,"[55022.53, 53382.86, 57862.89]","[64167.2, 59663.6, 60711.6]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,3589.61,3014.11,5.02,"[53936.22, 58630.69, 59533.57]","[59663.6, 60711.6, 60767.6]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,3916.02,3005.81,4.75,"[58965.96, 60005.61, 57760.49]","[60711.6, 60767.6, 64270.3]"
...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,1782.69,1433.92,6.15,"[23385.94, 22055.29, 20924.57]","[22349.7, 22437.4, 23808.0]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,2868.38,2454.73,10.58,"[21997.57, 20853.79, 18876.96]","[22437.4, 23808.0, 22847.1]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,3202.19,3152.09,13.84,"[20885.52, 18918.41, 19034.81]","[23808.0, 22847.1, 21639.9]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,2860.21,2788.67,12.95,"[19159.48, 19308.62, 17312.29]","[22847.1, 21639.9, 19659.4]"


Средние метрики Prophet по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,4692.09,4184.81,7.00
1,АКТЮБИНСКАЯ ОБЛАСТЬ,1595.96,1475.19,7.95
2,АЛМАТИНСКАЯ ОБЛАСТЬ,7727.78,7394.82,16.39
3,АТЫРАУСКАЯ ОБЛАСТЬ,2045.62,2028.18,82.17
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,1141.05,1121.65,20.98
5,ГАЛМАТЫ,8.33,6.19,38.15
6,ГАСТАНА,0.13,0.13,75.83
7,ГШЫМКЕНТ,909.73,779.78,5.36
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,1475.10,1396.37,13.42
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,1170.88,1032.42,8.40


In [9]:
# Переименуем колонки с MAPE, чтобы было понятно, к какому методу относятся
hw = final_hw.rename(columns={"MAPE_%": "MAPE_HW"})
sar = final_sarima.rename(columns={"MAPE_%": "MAPE_SARIMA"})
pr  = final_prophet.rename(columns={"MAPE_%": "MAPE_Prophet"})

# Мёрджим по региону
summary = (
    hw[["Регион", "MAPE_HW"]]
    .merge(sar[["Регион", "MAPE_SARIMA"]], on="Регион")
    .merge(pr[["Регион", "MAPE_Prophet"]], on="Регион")
)

# Определяем для каждой строки, какой столбец MAPE минимален
# idxmin вернёт название столбца с минимальным значением
summary["Best_method"] = summary[["MAPE_HW","MAPE_SARIMA","MAPE_Prophet"]] \
                           .idxmin(axis=1) \
                           .str.replace("MAPE_","")  # убираем префикс для красоты

# Если нужно, можно сразу отсортировать
# summary = summary.sort_values("Best_method")

# допустим, у вас уже есть summary
summary = summary.round({
    "MAPE_HW": 2,
    "MAPE_SARIMA": 2,
    "MAPE_Prophet": 2
})

# Готово!
print(summary.to_string(index=False))


                        Регион  MAPE_HW  MAPE_SARIMA  MAPE_Prophet Best_method
           АКМОЛИНСКАЯ ОБЛАСТЬ     6.27         6.72          7.00          HW
           АКТЮБИНСКАЯ ОБЛАСТЬ     5.93         6.20          7.95          HW
           АЛМАТИНСКАЯ ОБЛАСТЬ     7.25        11.34         16.39          HW
            АТЫРАУСКАЯ ОБЛАСТЬ    18.92        60.52         82.17          HW
ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ     4.91         6.32         20.98          HW
                       ГАЛМАТЫ    19.22        16.11         38.15      SARIMA
                       ГАСТАНА   144.40        89.75         75.83     Prophet
                      ГШЫМКЕНТ     5.94         8.99          5.36     Prophet
            ЖАМБЫЛСКАЯ ОБЛАСТЬ    17.52        18.91         13.42     Prophet
 ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ     6.87         6.90          8.40          HW
        КАРАГАНДИНСКАЯ ОБЛАСТЬ     4.94         5.90          9.55          HW
          КОСТАНАЙСКАЯ ОБЛАСТЬ    20.65        19.18